In [1]:
from spode.core import WaveGuide, PhaseShift, DirectionalCoupler, TBUm, GeneralModel, register_model
import numpy as np

ModuleNotFoundError: No module named 'const'

In [ ]:
omega = np.linspace(0, 10, 10)

In [ ]:
wg = WaveGuide(L=3., neff=2.)
wg.print_allattr()
S1 = wg.get_smatrix(omega)
print(S1.shape)
print('---')

In [ ]:
ps = PhaseShift(ps=0.1 * np.pi)
ps.print_allattr()
S2 = ps.get_smatrix(omega)
print(S2.shape)
print('---')

In [ ]:
dc = DirectionalCoupler(**{'cp_angle': 0.2 * np.pi})
dc.print_allattr()
S3 = dc.get_smatrix(omega)
print(S3.shape)
print('---')

In [ ]:
tbum = TBUm(**{'theta': 0., 'phi': 0., 'L': 250e-6, 'neff': 2.35})
tbum.print_allattr()
S4 = tbum.get_smatrix(omega)
print(S4.shape)
print('---')

In [2]:
# test a user-defined model
class TBUo(GeneralModel):
    """The class definition of a tunable basic unit (tbu) model with phase shifts in the middle and right, but in one row."""

    _name = 'tbuo'
    _required_attr = ['theta', 'phi', 'L', 'neff']
    _optional_attr = {'ng': 0, 'wl': None, 'alpha': 1.0, 'cp_left': 0.5 * np.pi, 'cp_right': 0.5 * np.pi}
    _differential_attr = ['theta', 'phi', 'L', 'cp_left', 'cp_right']
    _num_port = [2, 2]

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def get_smatrix(self,
                    omega: Union[list, np.ndarray],
                    direction: Optional[str] = None,
                    deri_vari: Optional[Union[str, List[str], Set[str]]] = None) -> \
            Union[Tuple[np.ndarray, dict], np.ndarray]:

        # linearly approximate the effective index neff
        neff_func = _neff(self.params['neff'], self.params['ng'], self.params['wl'])
        beta = omega * neff_func(omega) / const.FreeLightSpeed  # propagation constant

        # define and reshape the Smatrix to the correct shape (len(omega), self._num_port[0], self._num_port[1])

        S1 = np.zeros((2, 2), dtype=np.complex_)
        S1[0, 0] = S1[1, 1] = 1.0 * np.cos(self.params['cp_left'])
        S1[0, 1] = S1[1, 0] = 1.j * np.sin(self.params['cp_left'])

        S2 = np.zeros((2, 2), dtype=np.complex_)
        S2[0, 0] = np.exp(1.j * self.params['theta'])
        S2[1, 1] = np.exp(1.0)

        S3 = np.zeros((2, 2), dtype=np.complex_)
        S3[0, 0] = S3[1, 1] = 1.0 * np.cos(self.params['cp_right'])
        S3[0, 1] = S3[1, 0] = 1.j * np.sin(self.params['cp_right'])

        S4 = np.zeros((2, 2), dtype=np.complex_)
        S4[0, 0] = np.exp(1.j * self.params['phi'])
        S4[1, 1] = np.exp(1.0)
        
        # If the left and right directional couplers are not identical, then the S matrix will be different.
        if direction == 'r2l':
            S = S1 @ S2 @ S3 @ S4
            gradS_theta = S1 @ np.array([[1.j * np.exp(1.j * self.params['theta']), 0.], [0., 0.]]) @ S3 @ S4
            gradS_phi = S1 @ S2 @ S3 @ np.array([[1.j * np.exp(1.j * self.params['phi']), 0.], [0., 0.]])
        elif direction == 'l2r':
            S = S4 @ S3 @ S2 @ S1
            gradS_theta = S4 @ S3 @ np.array([[1.j * np.exp(1.j * self.params['theta']), 0.], [0., 0.]]) @ S1
            gradS_phi = np.array([[1.j * np.exp(1.j * self.params['phi']), 0.], [0., 0.]]) @ S3 @ S2 @ S1
        else:
            S = S4 @ S3 @ S2 @ S1
            gradS_theta = S4 @ S3 @ np.array([[1.j * np.exp(1.j * self.params['theta']), 0.], [0., 0.]]) @ S1
            gradS_phi = np.array([[1.j * np.exp(1.j * self.params['phi']), 0.], [0., 0.]]) @ S3 @ S2 @ S1
            # TODO: the warnign message printed is bit messy; could be improved.
            warnings.warn("The variable 'direction' should be either 'l2r' or 'r2l' when building S matrix. "
                          "Automatically use 'direction'='l2r'. This might lead to error, if the "
                          "two directional couplers in the {} are not identical.".format(self._name))

        S_wg = np.exp(1.j * beta * self.params['L']) * self.params['alpha']

        Smatrix = S_wg.reshape(-1, 1, 1) * np.expand_dims(S, axis=0).repeat(len(omega), axis=0)

        if deri_vari is not None:
            grad = {}
            if deri_vari.lower() == 'all':
                deri_vari = self._differential_attr

            if 'theta' in deri_vari:
                grad['theta'] = S_wg * gradS_theta

            if 'phi' in deri_vari:
                grad['phi'] = S_wg * gradS_phi

            if 'alpha' in deri_vari:
                grad['alpha'] = Smatrix / self.params['alpha']

            if 'L' in deri_vari:
                grad['L'] = Smatrix * 1.j * beta

            return Smatrix, grad
        else:
            return Smatrix

register_model(TBUo)

NameError: name 'GeneralModel' is not defined